This is jupyter notebook for teamates who are not familar with command line interface

In [2]:
import pandas as pd

In [4]:
path = '../data/haunted_places.tsv'
df= pd.read_csv(path, sep='\t')

In [15]:
# The number of unique cities
print(len(df.city.unique()))

4386


In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

##################################################################################3
# Source 1 : https://www.timeanddate.com/astronomy/usa

def webcrawl_daylight():
    # URL of interest
    url = "https://www.timeanddate.com/astronomy/usa"
    
    # 1. Fetch the page content
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve page, status code: {response.status_code}")
        return None, None

    # 2. Parse the HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 3. Find the target table (class="zebra fw tb-sm zebra")
    table = soup.find('table', {'class': 'zebra fw tb-sm zebra'})
    if not table:
        print("Could not find the expected table on the page.")
        return None, None

    data = []

    # 4. Loop over each row in the table (skipping the header)
    rows = table.find_all('tr')[1:]  # first row is the header
    for row in rows:
        cols = row.find_all('td')
        if len(cols) < 3:
            # Not a valid data row
            continue

        # Column 0: City/State info (e.g. "Adak (AK)")
        city_state_full = cols[0].get_text(strip=True)

        # Attempt to parse city and state. 
        # The format might be something like "Adak (AK)" or "Akron – Ohio – USA"
        # Adjust parsing logic according to actual structure you see on the site.
        # For demonstration, let's do a simple split:
        if " – " in city_state_full:
            parts = [p.strip() for p in city_state_full.split("–")]
            if len(parts) >= 2:
                city = parts[0]
                state = parts[1]
            else:
                city = city_state_full
                state = "Unknown"
        else:
            # If the string is like "Adak (AK)"
            city = city_state_full
            # Extract something in parentheses as state, e.g. (AK)
            # This is optional and depends on the site’s structure
            if "(" in city_state_full and ")" in city_state_full:
                state = city_state_full[city_state_full.index("(")+1 : city_state_full.index(")")]
                # Also remove the parentheses from city name
                city = city_state_full.split("(")[0].strip()
            else:
                state = "Unknown"

        # Column 1: Sunrise time (might include '↑')
        sunrise_str = cols[1].get_text(strip=True)
        # Column 2: Sunset time (might include '↓')
        sunset_str = cols[2].get_text(strip=True)

        # Remove unwanted symbols/characters
        for arrow in ['↑', '↓']:
            sunrise_str = sunrise_str.replace(arrow, '').strip()
            sunset_str = sunset_str.replace(arrow, '').strip()

        # 5. Parse times into datetime objects so we can compute the difference
        # Use a dummy date to parse times
        date_str = "2025-01-01"  
        
        try:
            sunrise_dt = datetime.strptime(date_str + " " + sunrise_str, "%Y-%m-%d %I:%M %p")
            sunset_dt = datetime.strptime(date_str + " " + sunset_str, "%Y-%m-%d %I:%M %p")
            # Compute difference in hours
            daylight_minutes = (sunset_dt - sunrise_dt).total_seconds() / 60
        except Exception as e:
            # If parsing fails, skip
            print(f"Parsing error for city={city}, sunrise={sunrise_str}, sunset={sunset_str}, error={e}")
            continue
        
        data.append({
            "city": city,
            "state": state,
            "sunrise": sunrise_dt.strftime("%I:%M %p"),
            "sunset": sunset_dt.strftime("%I:%M %p"),
            "daylight_minutes": daylight_minutes
        })

    # 6. Create a DataFrame
    df = pd.DataFrame(data)

    avg_major_cities = df['daylight_minutes'].mean()

    # 7. Calculate average daylight hours by State
    #avg_by_state = (
    #    df.groupby("state")["daylight_minutes"]
    #      .mean()
    #      .reset_index()
    #      .rename(columns={"daylight_minutes": "avg_daylight_minutes"})
    #)

    return df, avg_major_cities

def generate_daylight_avg_by_state():
    # avg_by_state has two columns: [state, avg_daylight_minutes]
    df, avg_daylight_of_major_cities = webcrawl_daylight()
    if df is not None and avg_daylight_of_major_cities is not None:
        print("\n--- Raw Data (City-Level) ---")
        print(df.head(20))

        print("\n--- Average Daylight Hours of major cities ---")
        #print(avg_daylight_of_major_cities.sort_values("avg_daylight_minutes", ascending=False))
    #avg_daylight_of_major_cities.to_csv('../data/daylight_s1')
    return round(avg_daylight_of_major_cities,1)

generate_daylight_avg_by_state()


--- Raw Data (City-Level) ---
           city state   sunrise    sunset  daylight_minutes
0          Adak    AK  08:21 AM  07:34 PM             673.0
1        Albany    NY  06:23 AM  05:49 PM             686.0
2   Albuquerque    NM  06:30 AM  06:06 PM             696.0
3          Ames    IA  06:42 AM  06:09 PM             687.0
4     Anchorage    AK  07:46 AM  06:36 PM             650.0
5     Annapolis    MD  06:32 AM  06:03 PM             691.0
6       Atlanta    GA  07:00 AM  06:37 PM             697.0
7       Augusta    ME  06:08 AM  05:32 PM             684.0
8        Austin    TX  06:52 AM  06:32 PM             700.0
9     Baltimore    MD  06:32 AM  06:03 PM             691.0
10  Baton Rouge    LA  06:25 AM  06:06 PM             701.0
11     Billings    MT  06:44 AM  06:06 PM             682.0
12     Bismarck    ND  07:14 AM  06:35 PM             681.0
13        Boise    ID  07:13 AM  06:39 PM             686.0
14       Boston    MA  06:12 AM  05:39 PM             687.0
15  Carso

686.2

In [34]:
temp = 5
x = {"A":[1,2,3,4],"B":[5,6,7,8]}
df = pd.DataFrame(x)
df["C"] = df["B"] - temp
df


,A,B,C
0,1,5,0
1,2,6,1
2,3,7,2
3,4,8,3
